**Exercise on the Quantum Classifier**

In [ ]:
pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.3 MB/s eta 0:00:00


**Example I: From Pennylane Demos > Fitting the parity function**

For this practice please make sure to download both parity_train.txt and parity_test.txt data from the following links:

https://drive.google.com/file/d/13ImDzaJpq1KhM9K4Ian3vfdczXR18EJg/view?usp=sharing

https://drive.google.com/file/d/1k3nTcsReZePZ0S19gXcsI8KKapFW0wyA/view?usp=sharing

In [ ]:
# Imports

import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

# Quantum and classical nodes

dev = qml.device("default.qubit")

def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])
@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


# Cost

def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labels - qml.math.stack(predictions)) ** 2)
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)


# Optimization

data = np.loadtxt("parity_train.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")


np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights:", weights_init)
print("Bias: ", bias_init)


opt = NesterovMomentumOptimizer(0.5)
batch_size = 5


weights = weights_init
bias = bias_init
for it in range(100):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")


data = np.loadtxt("parity_test.txt", dtype=int)
X_test = np.array(data[:, :-1])
Y_test = np.array(data[:, -1])
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)




x = [0 0 0 1], y = 1
x = [0 0 1 0], y = 1
x = [0 1 0 0], y = 1
x = [0 1 0 1], y = -1
x = [0 1 1 0], y = -1
x = [0 1 1 1], y = 1
x = [1 0 0 0], y = 1
x = [1 0 0 1], y = -1
x = [1 0 1 1], y = 1
x = [1 1 1 1], y = -1
Weights: [[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]
Bias:  0.0
Iter:    1 | Cost: 2.3147651 | Accuracy: 0.5000000
Iter:    2 | Cost: 1.9664866 | Accuracy: 0.5000000
Iter:    3 | Cost: 1.9208589 | Accuracy: 0.5000000
Iter:    4 | Cost: 2.6276126 | Accuracy: 0.5000000
Iter:    5 | Cost: 0.9323119 | Accuracy: 0.6000000
Iter:    6 | Cost: 1.1903549 | Accuracy: 0.5000000
Iter:    7 | Cost: 2.0508989 | Accuracy: 0.4000000
Iter:    8 | Cost: 1.1275531 | Accuracy: 0.6000000
Iter:    9 | Cost: 1.1659803 | Acc

**Part II: Practice**

**Part 1: Marketing**

 Advertising and Marketing
Customers may have various consumption profiles based on, for example, their age, frequency of purchases, intensity of their web store browsing (in minutes/day) and the budget spent on monthly purchases (in euros). This data can be used to determine whether a customer is likely to buy (or not) more items and thus send them (or not) promotional offers and discounts. Build the circuit that allows the mapping of a customer's consumption characteristics into quantum data to then decide whether or not to propose more advertising to a customer. Use for this the gate σ_{3} and a circuit depth Z equal to 2. You have to create the circuit using “IBM Quantum Composer” and deliver the OPENQASM code generated by the composer. Also, as a second option, you can use QISKIT to build the circuit. In both options, the deliverable would be a link to your Google Colab notebook.



**Part 2: Finance**

After the mapping of some economic data used to classify economies with finance at risk or not, we admit that the quantum data obtained after the mapping of quantum data produce the state |ψ> = α |0000> + β|1001> + ᵞ |0101> + σ|1111>. Build the circuit that corresponds to the classifier that allows the classification of this state with a depth equal to 2. You have to create the circuit using “IBM Quantum Composer” and deliver the OPENQASM code generated by the composer. Also, as a second option, you can use QISKIT to build the circuit. In both options, the deliverable would be a link to your Google Colab notebook.

**Part 3: Earthquake Classifier**

Natural phenomena can be catastrophic. It is therefore important to develop prevention systems to classify high-risk areas. As an example, several types of earthquakes exist: volcanic, human, etc. Regardless of their type, let us assume that an earthquake depends on the frequency of tectonic plate events, as well as volcanic activity and mining facilities. We want to develop a quantum variational classifier that takes these characteristics as input to know whether an area is at risk of earthquakes or not.
1. Construct the included quantum variational classifier, U_{Φ(X_{i})} and U_{Φ(W(Φ)} using the transformation σ_{1} and a circuit depth Z equal to 2 for the transformation U_{Φ(X_{i})} and a depth of 1 for U_{Φ(W(Φ)}?
2.   Run the circuit with an IBM simulator, indicating the output of the classifier C computes Ŷ when the input data are 40, 3, and 10 and Θ = Π/3? The output refers to class 0 (earthquake is unlikely to occur) or 1 (earthquake is unlikely to occur).
You have to create the circuit using “IBM Quantum Composer” and deliver the OPENQASM code generated by the composer. Also, as a second option, you can use QISKIT to build the circuit. In both options, the deliverable would be a link to your Google Colab notebook.
